In [ ]:
import numpy as np
import pandas as pd # Data processing
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns # Visualization lib based on matplotlib
from sklearn import preprocessing

color = sns.color_palette()
%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

from subprocess import check_output
print(check_output(["ls","../input"]).decode("UTF8"))

In [ ]:
print("### Files ###")
for f in os.listdir("../input"):
    print(f.ljust(30) + str(round(os.path.getsize("../input/"+f)/1000000,2)) + 'MB')

In [ ]:
df_train = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")

print("Train Data Shape: ", df_train.shape)
print("Test Data Shape: ", df_test.shape)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
y_train = df_train['y'].values

In [ ]:
plt.figure(figsize=(15,5))
plt.scatter(df_train['ID'].values,np.sort(y_train))
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(y_train,bins=20,kde=False)
plt.xlabel('y vlaues')
plt.ylabel('Occurences')
plt.title('y value distribution')
plt.show()

In [ ]:
print('min: {}, max: {}, mean: {}, std: {}'.format(min(y_train),max(y_train),y_train.mean(),y_train.std()))
print('Count of values above 175: {}'.format(np.sum(y_train>175)))

In [ ]:
df_train[df_train.columns[2:].values].dtypes.value_counts()

In [ ]:
catagorical_features = []
for c in df_train.columns[2:].values:
    if np.object == df_train[c].dtype:
        catagorical_features.append(c)          
print('catagorical_features: ',catagorical_features)

In [ ]:
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
label_encoder = preprocessing.LabelEncoder()
for f in catagorical_features:
    label_encoder.fit(list(df_train[f].values))
    df_train[f] = label_encoder.transform(list(df_train[f]))


In [ ]:
y_train = df_train['y'].values
x_train = df_train.drop(['ID','y'], axis=1)

In [ ]:
def xgb_r2_score(preds,dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels,preds)

In [ ]:
xgb_params = {
    'eta' : 0.02,
    'max_depth' : 6,
    'subsample' : 0.4,
    'colsample_bytree' : 0.4,
    'objective' : 'reg:linear',
    'silent' : 1
}

In [ ]:
dtrain = xgb.DMatrix(x_train, y_train, feature_names=x_train.columns.values)
model = xgb.train(dict(xgb_params,silent=1), dtrain, num_boost_round=100, feval=xgb_r2_score, maximize=True)

In [ ]:
fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(model, max_num_features=15, height=0.8, ax = ax)
plt.show()

In [ ]:
for f in catagorical_features:
    label_encoder.fit(list(df_test[f].values))
    df_test[f] = label_encoder.transform(list(df_test[f]))
x_test = df_test.drop(['ID'], axis=1)

In [ ]:
dtest = xgb.DMatrix(x_test)
y_test = model.predict(dtest)

In [ ]:
sub = pd.DataFrame()
sub['ID'] = df_test['ID']
sub['y'] = y_test

In [ ]:
sub.to_csv('sub_2.csv', index=False)